# 二分类问题与多层感知器

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 数据处理

In [2]:
# 读取数据
data = pd.read_csv('./dataset/HR.csv')
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,part,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
# 数据基本信息
print(data.info())
print(data.part.unique())
print(data.salary.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   part                   14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB
None
['sales' 'accounting' 'hr' 'technical' 'support' 'management' 'IT'
 'product_mng' 'marketing' 'RandD']
['low' 'medium' 'high']


In [4]:
# 数据预处理
# 简单的数据分析
# data.groupby(['salary', 'part']).size()
# .get_dummies()方法可以将分类数据转换为one-hot编码
data = data.join(pd.get_dummies(data.part).astype(int)).join(pd.get_dummies(data.salary).astype(int))
# 删除原来的分类数据
data.drop(columns=['part', 'salary'], inplace=True)
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,IT,RandD,...,hr,management,marketing,product_mng,sales,support,technical,high,low,medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   IT                     14999 non-null  int32  
 9   RandD                  14999 non-null  int32  
 10  accounting             14999 non-null  int32  
 11  hr                     14999 non-null  int32  
 12  management             14999 non-null  int32  
 13  marketing              14999 non-null  int32  
 14  product_mng            14999 non-null  int32  
 15  sa

In [6]:
# 查看离职率
print(data.left.value_counts())
# 全部预测为不离职
print(data.left.value_counts()[0] / data.left.value_counts().sum())

left
0    11428
1     3571
Name: count, dtype: int64
0.7619174611640777


In [7]:
# 处理结果数据
Y_data = data.left.values.reshape(-1, 1)
Y = torch.from_numpy(Y_data).type(torch.FloatTensor)
print(Y.shape)

torch.Size([14999, 1])


In [8]:
# 处理特征数据
# 使用列表推导式，获取除了'left'列之外的所有列
# [c for c in data.columns if c != 'left']
# 使用.values方法，将DataFrame转换为numpy数组
X_data = data[[c for c in data.columns if c != 'left']].values
X = torch.from_numpy(X_data).type(torch.FloatTensor)
print(X.shape)

torch.Size([14999, 20])


## 创建模型

In [9]:
# from torch import nn
# # 自定义模型：逻辑回归模型
# class Logistic(nn.Module):  # 继承nn.Module
#     def __init__(self):     # 初始化所有的层
#         super().__init__()  # 继承父类中所有的属性和方法
#         self.lin_1 = nn.Linear(20, 64)  # 定义第一层线性层，输入维度为20，输出维度为64
#         self.lin_2 = nn.Linear(64, 64)  # 定义第二层线性层，输入维度为64，输出维度为64
#         self.lin_3 = nn.Linear(64, 1)   # 定义第三层线性层，输入维度为64，输出维度为1
#         self.activate = nn.ReLU()       # 定义ReLU激活函数
#         self.sigmoid = nn.Sigmoid()     # 定义Sigmoid激活函数
#     def forward(self, input):   # 前向传播函数，定义模型的运算过程，覆盖父类中的forward方法
#         x = self.lin_1(input)   # 将输入数据传入第一层线性层
#         x = self.activate(x)    # ReLU激活函数
#         x = self.lin_2(x)       # 将激活后的数据传入第二层线性层
#         x = self.activate(x)    # ReLU激活函数
#         x = self.lin_3(x)       # 将激活后的数据传入第三层线性层
#         x = self.sigmoid(x)     # Sigmoid激活函数
#         return x

In [10]:
# 模型改写
from torch import nn
import torch.nn.functional as F # 函数式API，调用方便使代码更简洁
class Logistic(nn.Module):  # 继承nn.Module
    def __init__(self):     # 初始化所有的层
        super().__init__()  # 继承父类中所有的属性和方法
        self.lin_1 = nn.Linear(20, 64)  # 定义第一层线性层，输入维度为20，输出维度为64
        self.lin_2 = nn.Linear(64, 64)  # 定义第二层线性层，输入维度为64，输出维度为64
        self.lin_3 = nn.Linear(64, 1)   # 定义第三层线性层，输入维度为64，输出维度为1
    def forward(self, input):   # 前向传播函数，定义模型的运算过程，覆盖父类中的forward方法
        x = F.relu(self.lin_1(input))   # 将输入数据传入第一层线性层，并使用ReLU激活函数
        x = F.relu(self.lin_2(x))       # 将激活后的数据传入第二层线性层，并使用ReLU激活函数
        x = F.sigmoid(self.lin_3(x))     # 将激活后的数据传入第三层线性层，并使用Sigmoid激活函数
        return x

In [11]:
# 封装模型和优化器的创建，提高代码复用性
lr = 0.0001
def get_model():
    model = Logistic()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt
print(get_model())
model, opt = get_model()

(Logistic(
  (lin_1): Linear(in_features=20, out_features=64, bias=True)
  (lin_2): Linear(in_features=64, out_features=64, bias=True)
  (lin_3): Linear(in_features=64, out_features=1, bias=True)
), Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
))


## 模型训练

In [12]:
loss_fn = nn.BCELoss()

In [13]:
# 分割数据集，分批次进行训练
batch = 64
no_of_batches = len(data)//batch
epochs = 100

### 1. 手动分批次训练

In [14]:
# 分批次循环训练
for epoch in range(epochs):
    for i in range(no_of_batches):     # 按照批次进行训练
        start = i * batch              # 每个批次的起始索引
        end = start + batch            # 每个批次的结束索引
        x = X[start: end]
        y = Y[start: end]

        # Forward pass
        y_pred = model(x)
        # Compute loss: BCELoss expects the target to be between 0 and 1
        loss = loss_fn(y_pred, y)
        # Gradient reset
        opt.zero_grad()
        # Backward pass
        loss.backward()
        # Update the gradients
        opt.step()
    with torch.no_grad():
        print('epoch:', epoch, '   ', 'loss:', loss_fn(model(X), Y).data.item())

epoch: 0     loss: 0.6653244495391846
epoch: 1     loss: 0.6724502444267273
epoch: 2     loss: 0.66326904296875
epoch: 3     loss: 0.6563212871551514
epoch: 4     loss: 0.6486781239509583
epoch: 5     loss: 0.6963856816291809
epoch: 6     loss: 0.6340402364730835
epoch: 7     loss: 0.6262431144714355
epoch: 8     loss: 0.6186566352844238
epoch: 9     loss: 0.610680878162384
epoch: 10     loss: 0.624306857585907
epoch: 11     loss: 0.5956229567527771
epoch: 12     loss: 0.5905640721321106
epoch: 13     loss: 0.5859825611114502
epoch: 14     loss: 0.5818182826042175
epoch: 15     loss: 0.578037440776825
epoch: 16     loss: 0.5848634839057922
epoch: 17     loss: 0.5720465779304504
epoch: 18     loss: 0.5694150328636169
epoch: 19     loss: 0.5671862363815308
epoch: 20     loss: 0.5652644038200378
epoch: 21     loss: 0.5636811852455139
epoch: 22     loss: 0.5623966455459595
epoch: 23     loss: 0.5613612532615662
epoch: 24     loss: 0.580359935760498
epoch: 25     loss: 0.5601328015327454
ep

### 2. 使用dataset重构模型训练过程

PyTorch有一个抽象的Dataset类。Dataset可以是任何具有__len__函数和__getitem__作为对其进行索引的方法的函数。PyTorch的TensorDataset是一个包装张量的Dataset。通过定义索引的长度和方式，这也为我们提供了沿张量的第一维进行迭代，索引和切片的方法。这将使我们在训练的同一行中更容易访问自变量和因变量。下面将自定义HRDataset类创建为的Dataset的子类。

In [15]:
from torch.utils.data import TensorDataset
HRdataset = TensorDataset(X, Y)
# print(HRdataset[2: 5])
model, opt = get_model()

In [16]:
for epoch in range(epochs):
    for i in range(no_of_batches):
        x, y = HRdataset[i * batch: i * batch + batch]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        print('epoch:', epoch, '   ', 'loss:', loss_fn(model(X), Y).data.item())

epoch: 0     loss: 0.6955068111419678
epoch: 1     loss: 0.6839310526847839
epoch: 2     loss: 0.6755874752998352
epoch: 3     loss: 0.6657107472419739
epoch: 4     loss: 0.6551401615142822
epoch: 5     loss: 0.6444115042686462
epoch: 6     loss: 0.6337931752204895
epoch: 7     loss: 0.6235368847846985
epoch: 8     loss: 0.6138008236885071
epoch: 9     loss: 0.618353009223938
epoch: 10     loss: 0.6162864565849304
epoch: 11     loss: 0.6000204682350159
epoch: 12     loss: 0.5915645956993103
epoch: 13     loss: 0.5849820375442505
epoch: 14     loss: 0.5794337391853333
epoch: 15     loss: 0.5755189657211304
epoch: 16     loss: 0.5988996624946594
epoch: 17     loss: 0.5931522846221924
epoch: 18     loss: 0.5825529098510742
epoch: 19     loss: 0.575816810131073
epoch: 20     loss: 0.5719273686408997
epoch: 21     loss: 0.5671470165252686
epoch: 22     loss: 0.5641254782676697
epoch: 23     loss: 0.5622921586036682
epoch: 24     loss: 0.564382016658783
epoch: 25     loss: 0.5620140433311462

### 3. 使用DataLoader重构模型训练过程

Pytorch DataLoader负责管理批次，DataLoader从Dataset创建，自动为我们提供每个小批量，使遍历批次变得更容易，无需使用`HRdataset[i * batch: i * batch + batch]`

In [17]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

HRdataset = TensorDataset(X, Y)
HRdataloader = DataLoader(HRdataset, batch_size=batch) # batch_size: 每个批次的大小为，shuffle: 是否打乱数据

model, opt = get_model()

In [18]:
for epoch in range(epochs):
    for x, y in HRdataloader:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        print('epoch:', epoch, '   ', 'loss:', loss_fn(model(X), Y).data.item())

epoch: 0     loss: 0.7839525938034058
epoch: 1     loss: 0.8056097626686096
epoch: 2     loss: 0.8044900894165039
epoch: 3     loss: 0.786607563495636
epoch: 4     loss: 0.7772766947746277
epoch: 5     loss: 0.7672047019004822
epoch: 6     loss: 0.7565413117408752
epoch: 7     loss: 0.7444291114807129
epoch: 8     loss: 0.7335003614425659
epoch: 9     loss: 0.722275972366333
epoch: 10     loss: 0.7108554840087891
epoch: 11     loss: 0.6993186473846436
epoch: 12     loss: 0.6877512335777283
epoch: 13     loss: 0.6762359142303467
epoch: 14     loss: 0.664892852306366
epoch: 15     loss: 0.6529852151870728
epoch: 16     loss: 0.6387475728988647
epoch: 17     loss: 0.630553662776947
epoch: 18     loss: 0.6199312806129456
epoch: 19     loss: 0.6728217601776123
epoch: 20     loss: 0.6396542191505432
epoch: 21     loss: 0.621818482875824
epoch: 22     loss: 0.5924992561340332
epoch: 23     loss: 0.5756534934043884
epoch: 24     loss: 0.5921425223350525
epoch: 25     loss: 0.6098093390464783
e

## 划分验证数据集和测试数据集

**过拟合：** 指模型在训练数据上表现良好，但在验证数据（未知数据）上表现不佳。  
**欠拟合：** 指模型在训练数据上表现不佳，在验证数据上表现不佳。

前面我们只是试图建立一个合理的训练循环以用于我们的训练数据。实际上，始终还应该具有一个验证集，以识别是否过度拟合。

训练数据的乱序（shuffle）对于防止批次与过度拟合之间的相关性很重要。另一方面，无论我们是否乱序验证集，验证损失都是相同的。由于shufle需要额外的开销，因此shuffle验证数据没有任何意义。

我们将为验证集使用批大小，该批大小是训练集的两倍。这是因为验证集不需要反向传播，因此占用的内存更少（不需要存储梯度）。我们利用这一优势来使用更大的批量，并更快地计算损失。

In [19]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data) # 划分训练集和测试集
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(11249, 20) (11249, 1) (3750, 20) (3750, 1)


In [20]:
# 将numpy数组转换为PyTorch张量
train_x = torch.from_numpy(train_x).type(torch.FloatTensor)
test_x = torch.from_numpy(test_x).type(torch.FloatTensor)
train_y = torch.from_numpy(train_y).type(torch.FloatTensor)
test_y = torch.from_numpy(test_y).type(torch.FloatTensor)

# if torch.cuda.is_available():
#     train_x = train_x.to('cuda')
#     test_x = test_x.to('cuda')
#     train_y = train_y.to('cuda')
#     test_y = test_y.to('cuda')
#     print('Using GPU')

In [21]:
# 使用PyTorch的TensorDataset和DataLoader将数据集转换为数据加载器
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch)

valid_ds = TensorDataset(test_x, test_y)
valid_dl = DataLoader(valid_ds, batch_size=batch * 2)

# 计算正确率

In [22]:

# 准确率计算函数，用于计算预测值和真实值之间的准确率
def accuracy(y_pred, y_true):
    # 将预测值大于0.5的值转换为1，其余的转换为0
    y_pred = (y_pred>0.5).type(torch.IntTensor)
    # y_pred = (y_pred>0.5).type(torch.IntTensor).to('cuda')
    # 计算预测值和真实值相等的数量的均值，并转换为浮点数
    return (y_pred == y_true).float().mean()

In [23]:
epochs = 500
model, opt = get_model()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# if torch.cuda.is_available():
#     model = model.to('cuda')

for epoch in range(epochs):
    for x, y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y).data.item()
        epoch_loss = loss_fn(model(train_x), train_y).data.item()

        epoch_test_accuracy = accuracy(model(test_x), test_y).data.item()
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch:', epoch, '   ', 'loss:', round(epoch_loss, 3),
                               '   ', 'accuracy:', round(epoch_accuracy, 3),
                               '   ', 'test_loss:', round(epoch_test_loss, 3),
                               '   ', 'test_accuracy:', round(epoch_test_accuracy, 3))

epoch: 0     loss: 0.564     accuracy: 0.761     test_loss: 0.562     test_accuracy: 0.764
epoch: 1     loss: 0.56     accuracy: 0.761     test_loss: 0.559     test_accuracy: 0.764
epoch: 2     loss: 0.556     accuracy: 0.761     test_loss: 0.555     test_accuracy: 0.764
epoch: 3     loss: 0.552     accuracy: 0.761     test_loss: 0.55     test_accuracy: 0.764
epoch: 4     loss: 0.547     accuracy: 0.761     test_loss: 0.545     test_accuracy: 0.764
epoch: 5     loss: 0.541     accuracy: 0.761     test_loss: 0.54     test_accuracy: 0.764
epoch: 6     loss: 0.535     accuracy: 0.761     test_loss: 0.535     test_accuracy: 0.764
epoch: 7     loss: 0.528     accuracy: 0.761     test_loss: 0.527     test_accuracy: 0.764
epoch: 8     loss: 0.52     accuracy: 0.761     test_loss: 0.52     test_accuracy: 0.764
epoch: 9     loss: 0.512     accuracy: 0.761     test_loss: 0.512     test_accuracy: 0.764
epoch: 10     loss: 0.504     accuracy: 0.761     test_loss: 0.504     test_accuracy: 0.764
epo